In [10]:
import re
import pandas as pd
import plotly.express as px

from glob import glob

In [11]:
pt_time = re.compile(r"\[epoch (\d)\] elapsed time\: (\d+\.\d+) sec")
pt_p2p = re.compile(r"p2p_([a-z]+)")
pt_node = re.compile(r"node(\d+)")

In [12]:
logs = sorted(glob("./**/**/torch.log"))
rows = []

for log in logs:
    with open(log, "r") as f:
        lines = f.readlines()

    time_per_epoch = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
    for line in lines:
        sh_time = pt_time.search(line)
        if sh_time:
            time_per_epoch[int(sh_time.group(1))] = float(sh_time.group(2))

    p2p = pt_p2p.search(log).group(1)
    if p2p == "enable":
        p2p = "on"
    else:
        p2p = "off"

    node_num = int(pt_node.search(log).group(1))
    if node_num == 4:
        gpus = "A100 * 4"
    if node_num == 5:
        gpus = "H100 * 4"
    elif node_num == 7:
        gpus = "L40 * 4"
    elif node_num == 8:
        gpus = "A40 * 4"
    elif node_num == 9:
        gpus = "L4 * 4"

    rows.append(
        {
            "model": log.split("/")[1],
            "gpus": gpus,
            "nvlink": p2p,
            "epoch_1": round(time_per_epoch[1], 2),
            "epoch_2": round(time_per_epoch[2], 2),
            "epoch_3": round(time_per_epoch[3], 2),
            "epoch_4": round(time_per_epoch[4], 2),
            "epoch_5": round(time_per_epoch[5], 2),
        }
    )

In [13]:
df = pd.DataFrame(rows)
df.sort_values(by=["model", "gpus"], ascending=[False, True], inplace=True)
df.reset_index(drop=True, inplace=True)
df["mean"] = df.iloc[:, [3, 4, 5, 6, 7]].mean(axis=1)
df["standard_deviation"] = df.iloc[:, [3, 4, 5, 6, 7]].std(axis=1)
df.to_csv("data_nvlink.csv", index=False)
df

,model,gpus,nvlink,epoch_1,epoch_2,epoch_3,epoch_4,epoch_5,mean,standard_deviation
0,Llama-2-7b-chat-hf,A100 * 4,off,404.01,390.25,388.76,389.36,389.49,392.374,6.526303
1,Llama-2-7b-chat-hf,A100 * 4,on,44.63,29.40,29.41,29.43,29.38,32.450,6.808851
2,Llama-2-7b-chat-hf,H100 * 4,off,87.06,72.08,72.06,72.07,72.06,75.066,6.704855
3,Llama-2-7b-chat-hf,H100 * 4,on,28.47,13.50,13.51,13.72,13.63,16.566,6.655158
4,Llama-2-13b-chat-hf,A100 * 4,off,1475.53,1451.25,1451.08,1451.26,1451.62,1456.148,10.836654
5,Llama-2-13b-chat-hf,A100 * 4,on,165.55,150.96,143.52,155.61,152.53,153.634,8.008710
6,Llama-2-13b-chat-hf,H100 * 4,off,288.35,271.07,271.91,272.22,272.00,275.110,7.414233
7,Llama-2-13b-chat-hf,H100 * 4,on,49.65,31.10,31.07,30.97,31.05,34.768,8.319430
